### UNSUPERVISED MACHINE LEARNING FOR THE CLASSIFICATION OF ASTROPHYSICAL X-RAY SOURCES

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from astropy import stats
from astropy.io.votable import parse

import time
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
#plt.rcParams['figure.figsize'] = (16, 16)
#plt.style.use('ggplot')

In [2]:
def votable_to_pandas(votable_file):
    votable = parse(votable_file)
    table = votable.get_first_table().to_table(use_names_over_ids=True)
    return table.to_pandas()

In [3]:
data = votable_to_pandas("./data/corpus.vot")

---
# K-Means

In [4]:
features_out = ['name', 'ra', 'dec', 'theta', 'significance', 'src_area_b', 'flux_aper_b', 
                'hard_hm', 'hard_hs',
               'hard_ms', 'powlaw_gamma', 'bb_kt', 'var_prob_b', 'var_sigma_b',
               'var_mean_b', 'var_min_b', 'var_max_b', 'var_prob_h', 'var_sigma_h',
               'var_mean_h', 'var_min_h', 'var_max_h', 'var_prob_m', 'var_sigma_m',
               'var_mean_m', 'var_min_m', 'var_max_m', 'var_prob_s', 'var_sigma_s',
               'var_mean_s', 'var_min_s', 'var_max_s', 'ks_prob_b', 'ks_prob_h',
               'ks_prob_m', 'ks_prob_s', 'kp_prob_b', 'kp_prob_h', 'kp_prob_m',
               'kp_prob_s']

features = ['theta', 'significance', 'src_area_b', 'flux_aper_b', 
            'hard_hm', 'hard_hs',
           'hard_ms', 'powlaw_gamma', 'bb_kt', 'var_prob_b', 'var_sigma_b',
           'var_mean_b', 'var_min_b', 'var_max_b', 'var_prob_h', 'var_sigma_h',
           'var_mean_h', 'var_min_h', 'var_max_h', 'var_prob_m', 'var_sigma_m',
           'var_mean_m', 'var_min_m', 'var_max_m', 'var_prob_s', 'var_sigma_s',
           'var_mean_s', 'var_min_s', 'var_max_s', 'ks_prob_b', 'ks_prob_h',
           'ks_prob_m', 'ks_prob_s', 'kp_prob_b', 'kp_prob_h', 'kp_prob_m',
           'kp_prob_s']

features_lognorm = ['theta', 'significance', 'src_area_b', 'flux_aper_b', 
            'hard_hm', 'hard_hs',
           'hard_ms', 'bb_kt', 'var_prob_b', 'var_sigma_b',
           'var_mean_b', 'var_min_b', 'var_max_b', 'var_prob_h', 'var_sigma_h',
           'var_mean_h', 'var_min_h', 'var_max_h', 'var_prob_m', 'var_sigma_m',
           'var_mean_m', 'var_min_m', 'var_max_m', 'var_prob_s', 'var_sigma_s',
           'var_mean_s', 'var_min_s', 'var_max_s', 'ks_prob_b', 'ks_prob_h',
           'ks_prob_m', 'ks_prob_s', 'kp_prob_b', 'kp_prob_h', 'kp_prob_m',
           'kp_prob_s']

features_norm = ['powlaw_gamma']

X_df_out = data[features_out].dropna()
X_df = X_df_out[features]
X = X_df.copy().to_numpy()


In [5]:
# WHITENING
X_white = X_df.copy().to_numpy()
X_white = (X_white - np.mean(X_white))/np.std(X_white)

## Applying log transform and normalizing.

In [6]:
# FUNCTION lognorm
# Apply log transform adding the minimum non-zero value divided by ten in order to preserve zero properties, then normalize.
# INPUT:
# X_df = data array
# X = data array as np array
# name_desc = string, name of the descriptor
# log = boolean, True if apply log transform before norm

# PROCEDURE:
# Modifies X np array of data with the normalizated data
def lognorm(X_df, X, name_desc, log):
    
    col = X_df.columns.get_loc(name_desc)
    X_desc = X_df[name_desc]
    
    if log:
        nonzero = X_desc[X_desc!=0]
        minval = np.min(nonzero)/10

        # print(minval)
        X_desc = X_desc + minval

        x = np.log(X_desc.values)  #returns a numpy array
    else:
        x = X_desc.to_numpy()
    min_max_scaler = MinMaxScaler(feature_range=(0,1))
    x_scaled = min_max_scaler.fit_transform(x.reshape(-1,1))
    X[:,col] = x_scaled.flatten()
    
    return X

In [7]:
# Log transformation

for feature in features_lognorm:
    if np.max(X_df[feature]) > 2:
        X = lognorm(X_df, X, feature, True)
        
for feature in features_norm:
    if np.max(X_df[feature]) > 2:
        X = lognorm(X_df, X, feature, False)
        
# WHITENING
for feature in features_lognorm:
    if np.max(X_df[feature]) > 2:
        X_white = lognorm(X_df, X_white, feature, True)
        
for feature in features_norm:
    if np.max(X_df[feature]) > 2:
        X_white = lognorm(X_df, X_white, feature, False)

In [8]:
X_df = pd.DataFrame(X, columns=X_df.columns)
X_df = X_df.dropna()
X_df_out = X_df_out.dropna()
X = X_df.to_numpy()

In [9]:
X_white_df = pd.DataFrame(X_white, columns=X_df.columns)
X_white_df = X_white_df.dropna()
X_white = X_white_df.to_numpy()

---
# K-means iterative

In [10]:
def kmeans_iterative(X_df, X, n_cluster, n_it):
    iterations = []
    centroids = []
    for i in range(n_it):
        kmeans = KMeans(n_clusters=n_cluster).fit(X_df)
        C_current = kmeans.cluster_centers_
        centroids.append(np.linalg.norm(C_current))
        print('here',i)
        if i > 0:
            clust = []
            
            for x in C_current:
                dists = []
                for y in C_prev:
                    dist = np.linalg.norm(x-y)
                    dists.append(dist)
                mindist = min(dists)
                ind = np.where(dists == mindist)
                clust.append(dists[ind[0][0]])
            iterations.append(clust)
            
        C_prev = kmeans.cluster_centers_
            
    return iterations, centroids

In [11]:
iterations = []
centroids = []
t0 = time.time()
iterations, centroids = kmeans_iterative(X_df, X, 3, 50)
t1 = time.time()

total = t1-t0

print('time:',total)

here 0
here 1
here 2
here 3
here 4
here 5
here 6
here 7
here 8
here 9
here 10
here 11
here 12
here 13
here 14
here 15
here 16
here 17
here 18
here 19
here 20
here 21
here 22
here 23
here 24
here 25
here 26
here 27
here 28
here 29
here 30
here 31
here 32
here 33
here 34
here 35
here 36
here 37
here 38
here 39
here 40
here 41
here 42
here 43
here 44
here 45
here 46
here 47
here 48
here 49
time: 35.48857641220093


In [12]:
iterations = np.array(iterations)

In [13]:
iterations

array([[4.92136389e-04, 2.57080090e-04, 2.31080701e-04],
       [7.61277174e-04, 1.62909470e-04, 4.37653781e-04],
       [7.61277174e-04, 1.62909470e-04, 4.37653781e-04],
       [4.92136389e-04, 2.57080090e-04, 2.31080701e-04],
       [0.00000000e+00, 8.32667268e-17, 2.26339869e-16],
       [2.57080090e-04, 4.92136389e-04, 2.31080701e-04],
       [7.91713256e-04, 3.02879581e-03, 1.75030238e-03],
       [2.71738335e-03, 3.73081820e-03, 2.38223458e-03],
       [4.65904854e-03, 4.40488777e-03, 4.01103038e-03],
       [8.32667268e-17, 1.17961196e-16, 1.14439170e-16],
       [1.65372928e-16, 1.15277563e-16, 6.79869978e-17],
       [1.00074151e-16, 3.17980066e-17, 6.35960131e-17],
       [4.11888967e-04, 1.03182636e-03, 1.64944648e-03],
       [1.64944648e-03, 4.11888967e-04, 1.03182636e-03],
       [6.20633538e-17, 5.76387817e-17, 5.72195850e-17],
       [6.20633538e-17, 6.79869978e-17, 8.35553472e-17],
       [6.79869978e-17, 8.32667268e-17, 2.77555756e-17],
       [3.10316769e-17, 6.79869

In [14]:
iterations.mean()

0.0008490188321586746

In [15]:
iterations.std()

0.001406183114780649

In [16]:
iterations.var()

1.977350952294208e-06

In [18]:
np.array(centroids).mean()

5.352168817882301